In [1]:
import numpy as np
import pandas as pd
import tqdm

import peyes
import analysis.utils as u

## Load Data

In [2]:
DATASET = peyes.datasets.lund2013(directory=u.DATASETS_DIR, save=False, verbose=True)
IMAGE_DATASET = DATASET[DATASET[peyes.constants.STIMULUS_TYPE_STR] == peyes.constants.IMAGE_STR]

DATASET.head()

,trial_id,subject_id,stimulus_type,stimulus_name,t,x,y,pupil,pixel_size,viewer_distance,MN,RA
0,1,TH20,moving_dot,1,0.0,123.2532,22.6264,NaN,0.037824,67.0,1.0,1.0
1,1,TH20,moving_dot,1,2.0,123.5395,22.9064,NaN,0.037824,67.0,1.0,1.0
2,1,TH20,moving_dot,1,4.0,123.2230,21.9909,NaN,0.037824,67.0,1.0,1.0
3,1,TH20,moving_dot,1,6.0,123.1883,21.7740,NaN,0.037824,67.0,1.0,1.0
4,1,TH20,moving_dot,1,8.0,125.0540,21.1805,NaN,0.037824,67.0,1.0,1.0


## Algorithm-Specific Parameters
We extrapolate the "latent parameters" used by each rater as if they were implementing the detection algorithm during their annotations. Each algorithm has their own set of parameters to fit the annotated data.

### Engbert's $\lambda$ Parameter
The _Engbert_ algorithm uses a scalar parameter $\lambda$, to determine the SNR threshold differentiating saccades from fixations:  

\begin{gather*}
l_i = FIX \iff \frac{1}{\lambda^2} \cdot [(\frac{[v_x]_i}{<v_x>})^2 + (\frac{[v_y]_i}{<v_y>})^2] < 1
\end{gather*}

\begin{gather*}
l_i = SAC \iff \frac{1}{\lambda^2} \cdot [(\frac{[v_x]_i}{<v_x>})^2 + (\frac{[v_y]_i}{<v_y>})^2] \geq 1
\end{gather*}  

We denote $\vec{c}^2 = [(\frac{[v_x]_i}{<v_x>})^2 + (\frac{[v_y]_i}{<v_y>})^2]$, and we get:

\begin{gather*}
l_i = FIX \rightarrow (\frac{[\vec{c}]_i}{\lambda})^2 < 1 \rightarrow \lambda \geq [\vec{c}]_i
\end{gather*}

\begin{gather*}
l_i = SAC \rightarrow (\frac{[\vec{c}]_i}{\lambda})^2 \geq 1 \rightarrow \lambda \leq [\vec{c}]_i
\end{gather*}

Equivalently, we need to setisfy both conditions:

\begin{gather*}
l_i = FIX \rightarrow \lambda \geq \max([\vec{c}]_{\vec{l}\equiv FIX})
\end{gather*}

\begin{gather*}
l_i = SAC \rightarrow \lambda \leq \min([\vec{c}]_{\vec{l}\equiv SAC})
\end{gather*}


In [3]:
SR = 500
WINDOW_SIZE = 5
DEFAULT_LAMBDA = 5.0

def calculate_velocity_coeffs(x: np.ndarray, y: np.ndarray):
    vel_x = peyes._DataModels.Detector.EngbertDetector._axial_velocities_px(x, SR, WINDOW_SIZE)
    median_sd_x = peyes._DataModels.Detector.EngbertDetector._median_standard_deviation(vel_x)
    vel_y = peyes._DataModels.Detector.EngbertDetector._axial_velocities_px(y, SR, WINDOW_SIZE)
    median_sd_y = peyes._DataModels.Detector.EngbertDetector._median_standard_deviation(vel_y)
    coeff_squared = np.power(vel_x / median_sd_x, 2) + np.power(vel_y / median_sd_y, 2)
    return np.sqrt(coeff_squared)


def calculate_percentile_per_trial(trial_id: int, rater:str, lambda_: float) -> float:
    trial_data = IMAGE_DATASET[IMAGE_DATASET[peyes.constants.TRIAL_ID_STR] == trial_id]
    c = calculate_velocity_coeffs(
        trial_data[peyes.constants.X].values,
        trial_data[peyes.constants.Y].values
    )
    l = trial_data[rater].values
    fix_c = c[l == 1]
    sac_c = c[l == 2]
    percent_for_fixs = (fix_c <= lambda_).sum() / len(fix_c)
    percent_for_sacs = (sac_c >= lambda_).sum() / len(sac_c)
    return 100 * percent_for_fixs, 100 * percent_for_sacs


def calculate_success_rate(lambda_: float) -> pd.DataFrame:
    success_rates = {}
    for i, trial_id in tqdm.tqdm(enumerate(IMAGE_DATASET[peyes.constants.TRIAL_ID_STR].unique())):
        for rater in ["RA", "MN"]:
            fix_percent, sac_percent = calculate_percentile_per_trial(trial_id, rater, lambda_)
            if not np.isfinite(fix_percent) or not np.isfinite(sac_percent):
                continue
            success_rates[(trial_id, rater)] = (fix_percent, sac_percent)

    success_rates = pd.DataFrame(success_rates).T
    success_rates.columns = ["fix", "sac"]
    success_rates.index.names = ["trial_id", "annotator"]
    return success_rates

In [4]:
success_rates_df = pd.concat(
    {lambda_: calculate_success_rate(lambda_) for lambda_ in np.arange(1, 7.51, 0.5)},
    names=["lambda"]
)
success_rates_summary = success_rates_df.groupby(level=["lambda", "annotator"]).describe()
results = success_rates_summary[[
    ('fix', 'mean'), ('fix', 'std'), ('fix', '50%'), ('fix', 'min'), ('sac', 'mean'), ('sac', 'std'), ('sac', 'min'), ('sac', '50%')
]]
results

2it [00:00, 14.03it/s]C:\Users\jonathanni\AppData\Local\Temp\ipykernel_11984\1051624145.py:23: RuntimeWarning: invalid value encountered in scalar divide
  percent_for_fixs = (fix_c <= lambda_).sum() / len(fix_c)
C:\Users\jonathanni\AppData\Local\Temp\ipykernel_11984\1051624145.py:24: RuntimeWarning: invalid value encountered in scalar divide
  percent_for_sacs = (sac_c >= lambda_).sum() / len(sac_c)
20it [00:01, 16.76it/s]
2it [00:00, 15.71it/s]C:\Users\jonathanni\AppData\Local\Temp\ipykernel_11984\1051624145.py:23: RuntimeWarning: invalid value encountered in scalar divide
  percent_for_fixs = (fix_c <= lambda_).sum() / len(fix_c)
C:\Users\jonathanni\AppData\Local\Temp\ipykernel_11984\1051624145.py:24: RuntimeWarning: invalid value encountered in scalar divide
  percent_for_sacs = (sac_c >= lambda_).sum() / len(sac_c)
20it [00:01, 17.49it/s]
2it [00:00, 14.25it/s]C:\Users\jonathanni\AppData\Local\Temp\ipykernel_11984\1051624145.py:23: RuntimeWarning: invalid value encountered in scal

fix                                        sac  \
                       mean       std        50%        min       mean   
lambda annotator                                                         
1.0    MN         31.822183  4.823830  31.078700  23.245794  99.761945   
       RA         31.569222  5.080713  30.593547  23.449427  99.703634   
1.5    MN         53.854493  5.294417  53.614854  41.567501  99.508585   
       RA         53.895874  5.591484  52.357450  41.937023  99.447587   
2.0    MN         70.684502  4.726542  70.749191  59.253180  99.146429   
       RA         70.964463  4.970793  70.344697  59.661260  99.003942   
2.5    MN         81.478508  3.689269  81.351605  72.445630  98.801311   
       RA         81.968153  4.016855  81.781059  72.638359  98.495299   
3.0    MN         87.861496  2.767770  88.016762  82.191219  98.562192   
       RA         88.326734  3.302839  88.745267  82.299618  98.010180   
3.5    MN         91.666394  2.199297  92.074372  87.572182  98.140014   
       RA         92.086132  2.750741  92.369134  86.843487  97.438417   
4.0    MN         94.003493  1.785580  94.341795  90.007532  97.600151   
       RA         94.316237  2.246100  94.778704  89.233193  96.921149   
4.5    MN         95.522064  1.566136  95.820734  91.815215  97.168699   
       RA         95.758353  1.880539  96.378149  91.071429  96.391678   
5.0    MN         96.485741  1.468307  96.953446  92.844590  96.264129   
       RA         96.695100  1.670315  97.292370  92.148109  95.570938   
5.5    MN         97.176526  1.344356  97.453143  93.748431  95.770992   
       RA         97.358392  1.470326  97.881562  93.067227  94.931840   
6.0    MN         97.702106  1.202812  97.948496  94.677379  95.004070   
       RA         97.829188  1.250194  98.283199  94.012605  94.083149   
6.5    MN         98.042285  1.095014  98.331342  95.229726  94.344839   
       RA         98.188497  1.060209  98.595882  94.642857  93.395460   
7.0    MN         98.336363  0.982338  98.658122  95.857394  93.731429   
       RA         98.438378  0.917061  98.714856  95.430672  92.651479   
7.5    MN         98.609193  0.927011  98.875611  96.309315  92.832331   
       RA         98.684865  0.784022  98.924683  96.165966  91.764896   

                                                  
                       std        min        50%  
lambda annotator                                  
1.0    MN         0.282825  99.170124  99.873096  
       RA         0.411072  98.437500  99.907579  
1.5    MN         0.460268  98.555957  99.590989  
       RA         0.585056  97.656250  99.595412  
2.0    MN         0.600492  97.826087  99.411663  
       RA         0.822603  96.875000  99.074786  
2.5    MN         0.548249  97.826087  98.869979  
       RA         1.213482  95.121951  98.845436  
3.0    MN         0.622092  97.111913  98.657518  
       RA         1.674970  93.170732  98.544296  
3.5    MN         0.861419  95.667870  98.285169  
       RA         2.481057  88.780488  98.156011  
4.0    MN         1.120916  94.584838  97.518826  
       RA         2.745557  87.317073  97.781261  
4.5    MN         1.061688  94.584838  97.220284  
       RA         3.049399  85.853659  97.324674  
5.0    MN         1.565572  92.418773  96.476895  
       RA         3.553306  83.414634  96.443871  
5.5    MN         1.557239  92.057762  95.751398  
       RA         3.796980  82.439024  95.957252  
6.0    MN         1.691078  90.613718  95.543929  
       RA         4.266505  80.000000  95.287430  
6.5    MN         1.707002  90.613718  94.617560  
       RA         4.317863  79.512195  94.021107  
7.0    MN         2.135046  88.447653  94.056108  
       RA         4.668141  78.048780  93.724972  
7.5    MN         2.480938  87.003610  93.464730  
       RA         5.106993  76.585366  92.550618

In [5]:
results.xs("RA", level="annotator")

fix                                        sac            \
             mean       std        50%        min       mean       std   
lambda                                                                   
1.0     31.569222  5.080713  30.593547  23.449427  99.703634  0.411072   
1.5     53.895874  5.591484  52.357450  41.937023  99.447587  0.585056   
2.0     70.964463  4.970793  70.344697  59.661260  99.003942  0.822603   
2.5     81.968153  4.016855  81.781059  72.638359  98.495299  1.213482   
3.0     88.326734  3.302839  88.745267  82.299618  98.010180  1.674970   
3.5     92.086132  2.750741  92.369134  86.843487  97.438417  2.481057   
4.0     94.316237  2.246100  94.778704  89.233193  96.921149  2.745557   
4.5     95.758353  1.880539  96.378149  91.071429  96.391678  3.049399   
5.0     96.695100  1.670315  97.292370  92.148109  95.570938  3.553306   
5.5     97.358392  1.470326  97.881562  93.067227  94.931840  3.796980   
6.0     97.829188  1.250194  98.283199  94.012605  94.083149  4.266505   
6.5     98.188497  1.060209  98.595882  94.642857  93.395460  4.317863   
7.0     98.438378  0.917061  98.714856  95.430672  92.651479  4.668141   
7.5     98.684865  0.784022  98.924683  96.165966  91.764896  5.106993   

                              
              min        50%  
lambda                        
1.0     98.437500  99.907579  
1.5     97.656250  99.595412  
2.0     96.875000  99.074786  
2.5     95.121951  98.845436  
3.0     93.170732  98.544296  
3.5     88.780488  98.156011  
4.0     87.317073  97.781261  
4.5     85.853659  97.324674  
5.0     83.414634  96.443871  
5.5     82.439024  95.957252  
6.0     80.000000  95.287430  
6.5     79.512195  94.021107  
7.0     78.048780  93.724972  
7.5     76.585366  92.550618

In [6]:
results.xs("MN", level="annotator")

fix                                        sac            \
             mean       std        50%        min       mean       std   
lambda                                                                   
1.0     31.822183  4.823830  31.078700  23.245794  99.761945  0.282825   
1.5     53.854493  5.294417  53.614854  41.567501  99.508585  0.460268   
2.0     70.684502  4.726542  70.749191  59.253180  99.146429  0.600492   
2.5     81.478508  3.689269  81.351605  72.445630  98.801311  0.548249   
3.0     87.861496  2.767770  88.016762  82.191219  98.562192  0.622092   
3.5     91.666394  2.199297  92.074372  87.572182  98.140014  0.861419   
4.0     94.003493  1.785580  94.341795  90.007532  97.600151  1.120916   
4.5     95.522064  1.566136  95.820734  91.815215  97.168699  1.061688   
5.0     96.485741  1.468307  96.953446  92.844590  96.264129  1.565572   
5.5     97.176526  1.344356  97.453143  93.748431  95.770992  1.557239   
6.0     97.702106  1.202812  97.948496  94.677379  95.004070  1.691078   
6.5     98.042285  1.095014  98.331342  95.229726  94.344839  1.707002   
7.0     98.336363  0.982338  98.658122  95.857394  93.731429  2.135046   
7.5     98.609193  0.927011  98.875611  96.309315  92.832331  2.480938   

                              
              min        50%  
lambda                        
1.0     99.170124  99.873096  
1.5     98.555957  99.590989  
2.0     97.826087  99.411663  
2.5     97.826087  98.869979  
3.0     97.111913  98.657518  
3.5     95.667870  98.285169  
4.0     94.584838  97.518826  
4.5     94.584838  97.220284  
5.0     92.418773  96.476895  
5.5     92.057762  95.751398  
6.0     90.613718  95.543929  
6.5     90.613718  94.617560  
7.0     88.447653  94.056108  
7.5     87.003610  93.464730